<a href="https://colab.research.google.com/github/devashishbotre/Autonomous-Lane-Detector/blob/main/scripts/Video_Processing_and_Plotting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

manideep1108_tusimple_path = kagglehub.dataset_download('manideep1108/tusimple')
devashishbotre_models_path = kagglehub.dataset_download('devashishbotre/models')

print('Data source import complete.')


In [ ]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from tqdm import tqdm
from model_module import RESA, Decoder, LaneNet

IMG_HEIGHT, IMG_WIDTH = 720, 1280
NUM_CLASSES = 2

In [ ]:
def overlay_mask(image, mask, alpha=0.5):
    mask_colored = np.zeros_like(image)
    mask_colored[mask == 1] = [0, 255, 0]
    overlay = cv2.addWeighted(image, 1.0, mask_colored, alpha, 0.0)
    return overlay


def numeric_sort_key(name):
    return int(os.path.splitext(name)[0])

def process_directory(root_dir, model, device, output_dir="output_videos"):
    if not os.path.exists(root_dir):
        raise FileNotFoundError(f"Root directory {root_dir} does not exist")

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    original_video = cv2.VideoWriter(
        os.path.join(output_dir, "original_video_all.mp4"), fourcc, 20.0, (IMG_WIDTH, IMG_HEIGHT)
    )
    lane_video = cv2.VideoWriter(
        os.path.join(output_dir, "lane_detected_video_all.mp4"), fourcc, 20.0, (IMG_WIDTH, IMG_HEIGHT)
    )
    subdirs = sorted(
        [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))],
        key=lambda x: int(x)
    )

    total_frames_processed = 0

    for subdir in tqdm(subdirs, desc="Processing subdirectories"):
        subdir_path = os.path.join(root_dir, subdir)
        frames = sorted(
            [f for f in os.listdir(subdir_path) if f.endswith('.jpg')],
            key=numeric_sort_key
        )

        if len(frames) != 20:
            print(f"Warning: {subdir} has {len(frames)} frames, expected 20. Skipping.")
            continue

        for frame_name in frames:
            img_path = os.path.join(subdir_path, frame_name)
            image = cv2.imread(img_path)
            if image is None:
                print(f"Error: Failed to load {img_path}. Skipping.")
                continue

            image_resized = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
            image_rgb = cv2.cvtColor(image_resized, cv2.COLOR_BGR2RGB)
            image_tensor = torch.from_numpy(image_rgb.transpose(2, 0, 1)).float() / 255.0
            image_tensor = image_tensor.unsqueeze(0).to(device)

            model.eval()
            with torch.no_grad():
                output = model(image_tensor)
                mask = torch.argmax(output, dim=1).squeeze(0).cpu().numpy()

            overlay_image = overlay_mask(image_resized, mask)
            original_video.write(image_resized)
            lane_video.write(overlay_image)
            total_frames_processed += 1


    original_video.release()
    lane_video.release()
    print(f"Generated single videos: 'original_video_all.mp4' and 'lane_detected_video_all.mp4'")
    print(f"Total frames processed: {total_frames_processed}")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LaneNet(num_classes=NUM_CLASSES).to(device)
model.load_state_dict(torch.load("/kaggle/input/models/lane_model_final (2).pth", map_location=device,weights_only=True))
model.eval()

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 179MB/s]


LaneNet(
  (encoder): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [ ]:
root_dir = "/kaggle/input/tusimple/TUSimple/test_set/clips/0530"  # Adjust to your directory
process_directory(root_dir, model, device)

Processing subdirectories: 100%|██████████| 1248/1248 [34:28<00:00,  1.66s/it]

Generated single videos: 'original_video_all.mp4' and 'lane_detected_video_all.mp4'
Total frames processed: 24960
